In [1]:
from collections import Counter
import pandas as pd
import datetime
import matplotlib.pyplot as plt

wkdir = r"../../"

## A. Construct patent-level dataset.

### A1. Subset patents.

In [2]:
patent_df = pd.read_table(
    wkdir+r"data/raw/patentsview/g_patent.tsv", 
    dtype={'patent_id': str}, 
    usecols=["patent_id", "patent_type", "patent_date", "withdrawn"]
)
assert(len(patent_df)==len(set(patent_df.patent_id)))
print("Original number of patents: ", len(patent_df))
application_df = pd.read_table(
    wkdir+r"data/raw/patentsview/g_application.tsv",
    dtype={"patent_id": str, "patent_application_type": str, "series_code": str},
    usecols=["patent_id", "filing_date"]
)
assert(len(application_df)==len(set(application_df.patent_id)))
patent_df = patent_df[(patent_df.withdrawn==0) & (patent_df.patent_type=="utility")].merge(
    application_df, on="patent_id", how="inner"
)[["patent_id", "filing_date", "patent_date"]].rename(
    {"patent_date": "issue_date"}, axis=1
)
print("Number of utility patents with both application date and grant date and not withdrawn yet: ", len(patent_df))
print("Issue dates covered: from {} to {} ".format(min(patent_df.issue_date), max(patent_df.issue_date)))

Original number of patents:  8260142
Number of utility patents with both application date and grant date and not withdrawn yet:  7469992
Issue dates covered: from 1976-01-06 to 2022-09-27 


In [3]:
patent_df

,patent_id,filing_date,issue_date
0,10000000,2015-03-10,2018-06-19
1,10000001,2015-12-08,2018-06-19
2,10000002,2014-12-30,2018-06-19
3,10000003,2013-03-12,2018-06-19
4,10000004,2015-12-17,2018-06-19
...,...,...,...
7469987,RE31700,1982-11-30,1984-10-09
7469988,RE31701,1982-07-20,1984-10-09
7469989,RE31704,1982-05-10,1984-10-09
7469990,RE34357,1990-06-20,1993-08-24


In [4]:
location_df = pd.read_table(
    wkdir+r"data/raw/patentsview/g_location_disambiguated.tsv",
    dtype={"disambig_city": str, "disambig_state": str, "disambig_country": str},
    usecols=["location_id", "disambig_city", "disambig_state", "disambig_country"]
).rename(
    lambda x: x.replace("disambig_", "assignee_"), axis=1
)

In [5]:
location_df

,location_id,assignee_city,assignee_state,assignee_country
0,233cc488-16c8-11ed-9b5f-1234bde3cd05,Kabul,NaN,AF
1,8d61686e-16c8-11ed-9b5f-1234bde3cd05,Jalalabad,NaN,AF
2,4ce1cf12-16c8-11ed-9b5f-1234bde3cd05,Kunduz,NaN,AF
3,60eb1c9d-16c8-11ed-9b5f-1234bde3cd05,Balkh,NaN,AF
4,15c50d13-16c8-11ed-9b5f-1234bde3cd05,Sra,NaN,AF
...,...,...,...,...
81832,776f075b-16c8-11ed-9b5f-1234bde3cd05,Bulawayo,NaN,ZW
81833,7b46d8d3-16c8-11ed-9b5f-1234bde3cd05,Eiffel Flats,NaN,ZW
81834,7c598cc2-16c8-11ed-9b5f-1234bde3cd05,Macheke,NaN,ZW
81835,86cdfb85-16c8-11ed-9b5f-1234bde3cd05,Banket,NaN,ZW


In [6]:
assignee_df = pd.read_table(
    wkdir+r"data/raw/patentsview/g_assignee_disambiguated.tsv",
    dtype={"patent_id": str, "assignee_type": "Int64"}
)
print("Assignee types are tabulated as follows:")
display(pd.DataFrame.from_dict(
    Counter(assignee_df.assignee_type), orient='index'
).reset_index().rename(
    {"index": "type", 0: "count"}, axis=1
).sort_values("type").assign(
    description = lambda df: df.type.replace({
        1: "Unassigned", 2: "US Company", 3: "Foreign Company", 4: "US Individual", 5: "Foreign Individual", 
        6: "US Federal Government", 7: "Foreign Government", 8: "US County Government", 9: "US State Government",
        12: "US Company, Partial", 13: "Foreign Company, Partial", 14: "US Individual, Partial",
        15: "Foreign Individual, Partial", 17: "Foreign Government, Partial"
    })
).reset_index(drop=True))
assignee_df = assignee_df[assignee_df.assignee_type.isin([2, 3, 6, 7, 8, 9])]
print("Keep corporate and government assignees. The number of assignees per patent is distributed as follows:")
display(pd.DataFrame.from_dict(
    Counter(assignee_df.groupby("patent_id").size()), orient='index'
).reset_index().rename(
    {"index": "number of assignees per patent", 0: "count"}, axis=1
).sort_values("number of assignees per patent").reset_index(drop=True))
print("Keep only patents with one corporate assignee.")
assignee_df = assignee_df.merge(
    assignee_df.groupby("patent_id").size().pipe(lambda sr: sr[sr==1]).reset_index()["patent_id"], 
    on="patent_id", how="right"
)[
    ["patent_id", "assignee_id", "disambig_assignee_organization", "location_id"]
].rename({"disambig_assignee_organization": "assignee_name"}, axis=1).merge(
    location_df, on="location_id", how="left"
).drop("location_id", axis=1)

Assignee types are tabulated as follows:


,type,count,description
0,0,299,0
1,1,6,Unassigned
2,2,3641485,US Company
3,3,3767704,Foreign Company
4,4,42699,US Individual
5,5,32427,Foreign Individual
6,6,50765,US Federal Government
7,7,28538,Foreign Government
8,8,15,US County Government
9,9,1520,US State Government


Keep corporate and government assignees. The number of assignees per patent is distributed as follows:


,number of assignees per patent,count
0,1,7034250
1,2,197545
2,3,15385
3,4,2165
4,5,515
5,6,275
6,7,70
7,8,37
8,9,18
9,10,36


Keep only firms with one corporate assignee.


In [7]:
assignee_df

,patent_id,assignee_id,assignee_name,assignee_city,assignee_state,assignee_country
0,10000000,504d1121-c994-4dd3-a975-82998e69fba0,Raytheon Company,Waltham,MA,US
1,10000001,824bdded-49aa-41f7-8aff-8ad1ba3790ec,LS MTRON LTD.,Anyang-si,NaN,KR
2,10000002,f410c2f2-fc40-4a7f-97ba-7f4daa58587b,KOLON INDUSTRIES. INC.,Seocho-gu,NaN,KR
3,10000003,3a54eb86-6225-4650-8fef-ea1edf700514,KAUTEX TEXTRON GmbH & Co. KG,Bonn,NaN,DE
4,10000004,f1ed785b-8a42-4479-9e53-ee79bf1df54d,ZUBEX INDUSTRIAL SA DE CV,Monterrey,NaN,MX
...,...,...,...,...,...,...
7034245,T998008,7fc86005-eb2b-49f4-93da-79ff3679f422,International Business Machines Corporation,Armonk,NY,US
7034246,T998010,8a02dbca-74cd-4f5e-9604-bbc42400ce54,Continental Oil Company,Stamford,CT,US
7034247,T998012,60a424f6-0119-41e5-8eab-6be5008d5c25,"The United States of America, as represented b...",Washington,DC,US
7034248,T998013,60a424f6-0119-41e5-8eab-6be5008d5c25,"The United States of America, as represented b...",Washington,DC,US


In [8]:
patent_df = patent_df.merge(assignee_df, on="patent_id", how="inner")

In [9]:
patent_df

,patent_id,filing_date,issue_date,assignee_id,assignee_name,assignee_city,assignee_state,assignee_country
0,10000000,2015-03-10,2018-06-19,504d1121-c994-4dd3-a975-82998e69fba0,Raytheon Company,Waltham,MA,US
1,10000001,2015-12-08,2018-06-19,824bdded-49aa-41f7-8aff-8ad1ba3790ec,LS MTRON LTD.,Anyang-si,NaN,KR
2,10000002,2014-12-30,2018-06-19,f410c2f2-fc40-4a7f-97ba-7f4daa58587b,KOLON INDUSTRIES. INC.,Seocho-gu,NaN,KR
3,10000003,2013-03-12,2018-06-19,3a54eb86-6225-4650-8fef-ea1edf700514,KAUTEX TEXTRON GmbH & Co. KG,Bonn,NaN,DE
4,10000004,2015-12-17,2018-06-19,f1ed785b-8a42-4479-9e53-ee79bf1df54d,ZUBEX INDUSTRIAL SA DE CV,Monterrey,NaN,MX
...,...,...,...,...,...,...,...,...
6454061,RE31699,1983-05-12,1984-10-09,4d68a368-2e63-46df-92fd-8fe5b1cd6939,"ECKEL MANUFACTURING CO., INC.",Odessa,TX,US
6454062,RE31701,1982-07-20,1984-10-09,4fa2c0ee-ab8d-467a-9886-e741d66a8d8e,Kabushiki Kaisha Sato Kenkyusho,Kamifurano,NaN,JP
6454063,RE31704,1982-05-10,1984-10-09,daf58aec-32e8-45a0-947e-a4b39b7bf33e,"Litton Systems, Inc.",Beverly Hills,CA,US
6454064,RE34357,1990-06-20,1993-08-24,0501f7f5-4502-44b7-ad38-d14b621a1561,"ARMSTRONG WORLD INDUSTRIES, INC.",Lancaster,PA,US


### A2. Add citations, classification and inventor information.

In [10]:
inventor_df = pd.read_table(
    wkdir+r"data/raw/patentsview/g_inventor_disambiguated.tsv", 
    dtype={"patent_id": str}, 
    usecols=["patent_id", "inventor_sequence", "inventor_id", "disambig_inventor_name_first", "disambig_inventor_name_last"]
).rename(
    lambda x: x.replace("disambig_", ""), axis=1
).sort_values(
    ["patent_id", "inventor_sequence"]
)

In [11]:
inventor_df.to_pickle(wkdir+r"data/inProcess/raw_inventor_df.pkl")

In [12]:
inventor_df

,patent_id,inventor_sequence,inventor_id,inventor_name_first,inventor_name_last
0,10000000,0,fl:jo_ln:marron-5,Joseph,Marron
2,10000001,0,fl:su_ln:lee-389,Sun-Woo,Lee
1,10000001,1,fl:hy_ln:yu-30,Hyeon Jae,YU
6,10000002,0,fl:yu_ln:kim-48,Yun-Jo,Kim
5,10000002,1,fl:si_ln:kim-31,Si Min,Kim
...,...,...,...,...,...
20427561,T999002,0,2a3616tdt5rl235jmrrfoknyr,Henry C.,Twiggs
20427563,T999002,1,fl:ja_ln:suggs-2,James L.,Suggs
20427562,T999002,2,fl:da_ln:buck-13,Dan F.,Buck
20427565,T999003,0,fl:jo_ln:ford-7,John A.,Ford


In [13]:
cites_df = pd.read_table(
    wkdir+r"data/raw/patentsview/g_us_patent_citation.tsv", 
    dtype={"patent_id": str, "citation_patent_id": str},
    usecols=["patent_id", "citation_patent_id"]
).merge(
    patent_df[["patent_id", "issue_date"]].rename(lambda name: f"citation_{name}", axis=1),
    on="citation_patent_id", how="right"
).merge(
    patent_df[["patent_id", "issue_date"]], 
    on="patent_id", how="left"
).assign(
    within_five_year = lambda df: pd.to_datetime(df.issue_date) - pd.to_datetime(df.citation_issue_date) <= pd.Timedelta(days=5*365+1)
).groupby(["citation_patent_id"]).agg(
    cites = ("patent_id", "count"),
    cites_5yr = ("within_five_year", "sum"),
    issue_date = ("citation_issue_date", "last")
).assign(
    issue_year = lambda df: df.issue_date.apply(lambda x: int(x[:4])),
    cites_dfe = lambda df: df.cites / df.groupby("issue_year")["cites"].transform("sum") * 1e6,
    cites_5yr_dfe = lambda df: df.cites_5yr / df.groupby("issue_year")["cites_5yr"].transform("sum") * 1e6
).reset_index().rename(columns={"citation_patent_id": "patent_id"})

In [14]:
cites_df

,patent_id,cites,cites_5yr,issue_date,issue_year,cites_dfe,cites_5yr_dfe
0,10000000,3,3,2018-06-19,2018,7.171629,7.792430
1,10000001,0,0,2018-06-19,2018,0.000000,0.000000
2,10000002,0,0,2018-06-19,2018,0.000000,0.000000
3,10000003,1,1,2018-06-19,2018,2.390543,2.597477
4,10000004,0,0,2018-06-19,2018,0.000000,0.000000
...,...,...,...,...,...,...,...
6454061,RE31699,30,0,1984-10-09,1984,28.620192,0.000000
6454062,RE31701,0,0,1984-10-09,1984,0.000000,0.000000
6454063,RE31704,5,0,1984-10-09,1984,4.770032,0.000000
6454064,RE34357,8,0,1993-08-24,1993,3.510937,0.000000


In [15]:
cites_df.to_pickle(wkdir+r"data/inProcess/raw_cites_df.pkl")

In [18]:
# Forward citation data.
patent_df = patent_df.merge(
    cites_df[["patent_id", "cites", "cites_5yr", "cites_dfe", "cites_5yr_dfe"]],
    on="patent_id", how="left"
).fillna({"cites": 0, "cites_5yr": 0, "cites_dfe": 0, "cites_5yr_dfe": 0}).astype(
    {"cites": "int64", "cites_5yr": "int64", "cites_dfe": "float", "cites_5yr_dfe": "float64"}
)

In [19]:
# CPC classification data.
patent_df = patent_df.merge(
    pd.read_table(
        wkdir+r"data/raw/patentsview/g_cpc_current.tsv", 
        dtype={"patent_id": str},
        usecols=["patent_id", "cpc_sequence", "cpc_group"]
    ).sort_values(
        ["patent_id", "cpc_sequence"]
    ).groupby("patent_id").apply(
        lambda df: ';'.join(df.cpc_group)
    ).rename("cpc_groups").reset_index(), 
    on="patent_id", how="left"
)

In [20]:
# IPC classification data.
patent_df = patent_df.merge(
    pd.read_table(
        wkdir+r"data/raw/patentsview/g_ipc_at_issue.tsv", 
        dtype={"patent_id": str, "ipc_class": str, "main_group": str},
        usecols=["patent_id", "ipc_sequence", "section", "ipc_class", "subclass", "main_group"]
    ).dropna().sort_values(
        ["patent_id", "ipc_sequence"]
    ).assign(
        ipc_group = lambda df: df.section + df.ipc_class + df.subclass + df.main_group
    )[["patent_id", "ipc_group"]].drop_duplicates().groupby("patent_id").apply(
        lambda df: ';'.join(df.ipc_group)
    ).rename("ipc_groups").reset_index(), 
    on="patent_id", how="left"
)

In [21]:
# USPC classification data.
patent_df = patent_df.merge(
    pd.read_table(
        wkdir+r"data/raw/patentsview/g_uspc_at_issue.tsv",
        dtype={"patent_id": str},
        usecols=["patent_id", "uspc_sequence", "uspc_subclass_id"]
    ).sort_values(
        ["patent_id", "uspc_sequence"]
    ).groupby("patent_id").apply(
        lambda df: ';'.join(df.uspc_subclass_id)
    ).rename("uspc_subclasses").reset_index(), 
    on="patent_id", how="left"
)

In [22]:
# Inventor list.
patent_df = patent_df.merge(
    inventor_df.groupby("patent_id").apply(
        lambda df: ';'.join(df.inventor_id)
    ).rename("inventors_id").reset_index(),
    on="patent_id", how="left"
)

In [23]:
# KPSS data.
patent_df = patent_df.merge(
    pd.read_csv(
        wkdir+r"data/raw/kpss/KPSS_2020_public.csv",
        dtype={"patent_num": str},
        usecols=["patent_num", "xi_real", "xi_nominal", "cites", "permno"]
    ).rename({
        "patent_num": "patent_id",
        "xi_real": "kpss_xi_real",
        "xi_nominal": "kpss_xi_nominal",
        "cites": "kpss_cites",
        "permno": "kpss_permno"
    }, axis=1),
    on="patent_id", how="left"
)

In [24]:
print(f"Among {len(patent_df)} patents, {sum(~patent_df.kpss_permno.isna())} are covered by KPSS (extended to 2020).")

Among 6454066 patents, 2359806 are covered by KPSS (extended to 2020).


In [25]:
# AERI data.
patent_df = patent_df.merge(
    pd.read_csv(
        wkdir+r"data/raw/aeri/PatentSimilarityImportanceBreakthrough_forPost2022.csv",
        dtype={"patent_num": str},
        usecols=["patent_num", "bsim5", "fsim01", "fsim25", "fsim610", "lqsim05", "lqsim010", "bk_p90_alqsim05", "bk_p90_alqsim010", "fcitALL"]
    ).rename(columns=lambda x: "aeri_"+x if x!="patent_num" else "patent_id"),
    on="patent_id", how="left"
)

In [26]:
print(f"Among {len(patent_df)} patents, {sum(~patent_df.aeri_fcitALL.isna())} are covered by AERI (extended to 2022).")

Among 6454066 patents, 6453097 are covered by AERI (extended to 2022).


In [30]:
patent_df.to_csv(wkdir+r"data/inProcess/patent_all_corp.csv", index=False)

In [31]:
patent_df.to_pickle(wkdir+r"data/inProcess/patent_all_corp.pkl")

In [29]:
patent_df.columns

Index(['patent_id', 'filing_date', 'issue_date', 'assignee_id',
       'assignee_name', 'assignee_city', 'assignee_state', 'assignee_country',
       'cites', 'cites_5yr', 'cites_dfe', 'cites_5yr_dfe', 'cpc_groups',
       'ipc_groups', 'uspc_subclasses', 'inventors_id', 'kpss_xi_real',
       'kpss_xi_nominal', 'kpss_cites', 'kpss_permno', 'aeri_bsim5',
       'aeri_fsim01', 'aeri_fsim25', 'aeri_fsim610', 'aeri_lqsim05',
       'aeri_lqsim010', 'aeri_bk_p90_alqsim05', 'aeri_bk_p90_alqsim010',
       'aeri_fcitALL'],
      dtype='object')

In [27]:
patent_df

,patent_id,filing_date,issue_date,assignee_id,assignee_name,assignee_city,assignee_state,assignee_country,cites,cites_5yr,...,kpss_permno,aeri_bsim5,aeri_fsim01,aeri_fsim25,aeri_fsim610,aeri_lqsim05,aeri_lqsim010,aeri_bk_p90_alqsim05,aeri_bk_p90_alqsim010,aeri_fcitALL
0,10000000,2015-03-10,2018-06-19,504d1121-c994-4dd3-a975-82998e69fba0,Raytheon Company,Waltham,MA,US,3,3,...,24942.0,16137348.0,3717139.0,13095141.0,NaN,NaN,NaN,NaN,NaN,8.0
1,10000001,2015-12-08,2018-06-19,824bdded-49aa-41f7-8aff-8ad1ba3790ec,LS MTRON LTD.,Anyang-si,NaN,KR,0,0,...,NaN,4491978.0,1061785.0,3338164.0,NaN,NaN,NaN,NaN,NaN,0.0
2,10000002,2014-12-30,2018-06-19,f410c2f2-fc40-4a7f-97ba-7f4daa58587b,KOLON INDUSTRIES. INC.,Seocho-gu,NaN,KR,0,0,...,NaN,15684418.0,3809230.0,14191477.0,NaN,NaN,NaN,NaN,NaN,0.0
3,10000003,2013-03-12,2018-06-19,3a54eb86-6225-4650-8fef-ea1edf700514,KAUTEX TEXTRON GmbH & Co. KG,Bonn,NaN,DE,1,1,...,23579.0,2376130.0,614547.0,2407175.0,NaN,NaN,NaN,NaN,NaN,2.0
4,10000004,2015-12-17,2018-06-19,f1ed785b-8a42-4479-9e53-ee79bf1df54d,ZUBEX INDUSTRIAL SA DE CV,Monterrey,NaN,MX,0,0,...,NaN,6720278.0,1472208.0,4603263.0,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6454061,RE31699,1983-05-12,1984-10-09,4d68a368-2e63-46df-92fd-8fe5b1cd6939,"ECKEL MANUFACTURING CO., INC.",Odessa,TX,US,30,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6454062,RE31701,1982-07-20,1984-10-09,4fa2c0ee-ab8d-467a-9886-e741d66a8d8e,Kabushiki Kaisha Sato Kenkyusho,Kamifurano,NaN,JP,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6454063,RE31704,1982-05-10,1984-10-09,daf58aec-32e8-45a0-947e-a4b39b7bf33e,"Litton Systems, Inc.",Beverly Hills,CA,US,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6454064,RE34357,1990-06-20,1993-08-24,0501f7f5-4502-44b7-ad38-d14b621a1561,"ARMSTRONG WORLD INDUSTRIES, INC.",Lancaster,PA,US,8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## B. Construct inventor-level data.

In [34]:
# Link inventors to assignees (firms) through patents.
inventor_assignee_df = inventor_df.merge(
    patent_df[["patent_id", "assignee_id", "filing_date", "cites", "kpss_cites", "kpss_xi_real"]],
    on="patent_id", how="inner"
).assign(
    filing_year = lambda df: df.filing_date.apply(lambda x: int(x[:4])),
    team_size = lambda df: df.groupby("patent_id")["patent_id"].transform(len),
    cites_pc = lambda df: df.cites / df.team_size,
    kpss_cites_pc = lambda df: df.kpss_cites / df.team_size,
    kpss_xi_real_pc = lambda df: df.kpss_xi_real / df.team_size
).pipe(
    lambda df: df[(df.filing_year>1950) & (df.filing_year<=datetime.datetime.now().year)]
).sort_values(
    ["inventor_id", "filing_date"]
).groupby(["inventor_id", "filing_year", "assignee_id"]).agg(
    # Aggregate to inventor-filing_year-assignee level.
    inventor_name_first = ("inventor_name_first", "last"),
    inventor_name_last = ("inventor_name_last", "last"),
    patent_count = ("patent_id", "count"),
    last_filing_date = ("filing_date", "last"),
    patent_count_cites_weighted = ("cites", "sum"),
    patent_count_cites_pc_weighted = ("cites_pc", "sum"),
    kpss_count = ("kpss_cites", "count"),
    patent_count_kpss_cites_weighted = ("kpss_cites", "sum"),
    patent_count_kpss_cites_pc_weighted = ("kpss_cites_pc", "sum"),
    patent_count_kpss_xi_real_weighted = ("kpss_xi_real", "sum"),
    patent_count_kpss_xi_real_pc_weighted = ("kpss_xi_real_pc", "sum")
).reset_index().assign(**{
    col: lambda df: df[col].where(df.kpss_count > 0)
    for col in [
        "patent_count_kpss_cites_weighted", "patent_count_kpss_cites_pc_weighted",
        "patent_count_kpss_xi_real_weighted", "patent_count_kpss_xi_real_pc_weighted"
    ]
}).drop("kpss_count", axis=1)

Generate both output measures, one divided by team size and one not.

In [35]:
display(pd.DataFrame.from_dict(Counter(
    inventor_df.merge(
        patent_df[["patent_id", "assignee_id", "filing_date", "cites", "kpss_cites", "kpss_xi_real"]],
        on="patent_id", how="inner"
    ).assign(
        filing_year = lambda df: df.filing_date.apply(lambda x: int(x[:4]))
    ).pipe(
        lambda df: df[(df.filing_year>1950) & (df.filing_year<=datetime.datetime.now().year)]
    ).groupby(["inventor_id", "filing_year"])["assignee_id"].nunique()), 
    orient='index'
).reset_index().rename(
    {"index": "number of assignees per inventor-year", 0: "count"}, axis=1
).sort_values("number of assignees per inventor-year").reset_index(drop=True).head(10))

,number of assignees per inventor-year,count
0,1,9111059
1,2,598024
2,3,84488
3,4,21068
4,5,7791
5,6,3529
6,7,1904
7,8,975
8,9,555
9,10,368


When determining the affiliation of an inventor in a year, use the latest mode among all assignees.

In [39]:
inventor_assignee_df = pd.DataFrame(
    # Generate inventor-year pairs.
    [(row.inventor_id, year) for _, row in inventor_assignee_df.groupby("inventor_id").agg(
        filing_year_first = ("filing_year", "first"),
        filing_year_last = ("filing_year", "last")
    ).reset_index().iterrows() for year in range(row.filing_year_first, row.filing_year_last+1)],
    columns=["inventor_id", "filing_year"]
).merge(
    # Add assignee info when available.
    inventor_assignee_df.sort_values(
        ["inventor_id", "filing_year", "patent_count", "last_filing_date"]
    ).groupby(
        ["inventor_id", "filing_year"]
    )["assignee_id"].last().reset_index(), 
    on=["inventor_id", "filing_year"], how="left"
).assign(
    # Forward fill missing assignees.
    assignee_id = lambda df: df.groupby("inventor_id")["assignee_id"].ffill(),
    affiliation_mark = True
).merge(
    # Merge more patent info.
    inventor_assignee_df, on=["inventor_id", "filing_year", "assignee_id"], how="outer"
).drop("last_filing_date", axis=1).fillna({"affiliation_mark": False}).rename({"filing_year": "year"}, axis=1)

In [40]:
inventor_assignee_df.to_csv(wkdir+r"data/inProcess/inventor_all_corp.csv", index=False)
inventor_assignee_df.to_pickle(wkdir+r"data/inProcess/inventor_all_corp.pkl")

Note that inventor-year pairs might not be unique in this dataset. For some years, there might be multiple assignees.

In [41]:
inventor_assignee_df

,inventor_id,year,assignee_id,affiliation_mark,inventor_name_first,inventor_name_last,patent_count,patent_count_cites_weighted,patent_count_cites_pc_weighted,patent_count_kpss_cites_weighted,patent_count_kpss_cites_pc_weighted,patent_count_kpss_xi_real_weighted,patent_count_kpss_xi_real_pc_weighted
0,0000n8nqsxhrztn7djlxou00k,1999,ee410ae3-ad00-44e3-8182-d4e45e9ebe91,True,Muamer,Zukic,1.0,17.0,3.400000,NaN,NaN,NaN,NaN
1,000f0k6brgval6kr9agzjlgcg,1990,7fc86005-eb2b-49f4-93da-79ff3679f422,True,Lynda M.,Salvetti,1.0,30.0,3.750000,0.995260,0.995260,0.995260,0.995260
2,000fjkbab9nljqyqpy20ktebx,1997,0ef48bf0-2634-4f1f-bc72-43ed9cb94617,True,Seved Ahmad Mojahed,Gomnam,1.0,23.0,5.750000,NaN,NaN,NaN,NaN
3,000jdp4dd6gu6ah3gvb1u0biq,1996,255f3b0c-138c-4742-ac27-f3dcfd865d3b,True,Hiroshi,Kuwako,1.0,3.0,0.375000,NaN,NaN,NaN,NaN
4,000s9qy0x4l3x4n99bv2a1qn4,2000,504d1121-c994-4dd3-a975-82998e69fba0,True,April A.,Barbour,1.0,12.0,6.000000,7.271756,7.271756,7.271756,7.271756
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21973208,fl:zy_ln:haas-1,1995,928e0323-a839-43a5-b4e5-38b6c21301d2,False,Zygmunt,Haas,1.0,31.0,15.500000,4.529985,4.529985,4.529985,4.529985
21973209,fl:zy_ln:haas-1,1998,1ec61c0b-6d97-43bf-9f36-0cb64368cc0e,False,Zygmunt,Haas,1.0,551.0,551.000000,NaN,NaN,NaN,NaN
21973210,fl:zy_ln:madrzak-2,1998,2b8f3ce8-e6de-4bdf-9a20-bfe7dbac7f8d,False,Zygmunt,Madrzak,1.0,18.0,3.600000,NaN,NaN,NaN,NaN
21973211,fl:zy_ln:madrzak-2,1999,15046867-ae0a-4036-b057-d0762bcca904,False,Zygmunt,Madrzak,1.0,5.0,0.555556,NaN,NaN,NaN,NaN


## C. Construct firm-level data.

In [42]:
firm_df = patent_df.assign(
    year = lambda df: df.filing_date.apply(lambda x: int(x[:4]))
).groupby(["assignee_id", "year"]).agg(
    patent_count = ("patent_id", "count"),
    cites_total = ("cites", "sum"),
    kpss_xi_real_total = ("kpss_xi_real", "sum"),
    kpss_cites_total = ("kpss_cites", "sum"),
    kpss_permno = ("kpss_permno", "last")
).merge(
    inventor_assignee_df.groupby(["assignee_id", "year"]).agg(
        inventor_count = ("inventor_id", "count"),
        inventor_count_affiliated = ("affiliation_mark", "sum")
    ), on=["assignee_id", "year"], how="right"
).reset_index()

In [43]:
firm_df.to_csv(wkdir+r"data/inProcess/firm_all_corp.csv", index=False)
firm_df.to_pickle(wkdir+r"data/inProcess/firm_all_corp.pkl")

In [44]:
firm_df

,assignee_id,year,patent_count,cites_total,kpss_xi_nominal_total,kpss_cites_total,kpss_permno,inventor_count,inventor_count_affiliated
0,000062dc-74ee-4de1-bbb6-84bcf6ef7df2,1992,1.0,6.0,0.125159,5.0,11485.0,2,2
1,00008134-bacb-4db3-a0fc-217512861277,2005,1.0,4.0,0.000000,0.0,NaN,4,0
2,00008134-bacb-4db3-a0fc-217512861277,2006,1.0,0.0,0.000000,0.0,NaN,6,0
3,00008134-bacb-4db3-a0fc-217512861277,2018,1.0,0.0,0.000000,0.0,NaN,6,0
4,0000d794-7de9-47ad-a514-5fc4eaa5a1d9,2020,1.0,0.0,0.000000,0.0,NaN,1,1
...,...,...,...,...,...,...,...,...,...
2690744,ffff43cd-6dae-4bd9-832f-70a7f18b5172,1993,NaN,NaN,NaN,NaN,NaN,1,1
2690745,ffff43cd-6dae-4bd9-832f-70a7f18b5172,1994,NaN,NaN,NaN,NaN,NaN,1,1
2690746,ffff80a8-d478-4187-93dc-1b712dfb8cd7,2017,2.0,0.0,0.000000,0.0,NaN,3,3
2690747,ffff80a8-d478-4187-93dc-1b712dfb8cd7,2018,1.0,0.0,0.000000,0.0,NaN,2,2
